# Connected components

-----

## Setup

Here are loaded the packages used in the rest of notebook:

In [57]:
use Graph;
use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Wheel;
use Graph::Path;
use Graph::Petersen;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Math::DistanceFunctions;
use Statistics::Distributions;

use JavaScript::D3;
use WWW::MermaidInk;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [58]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [59]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [60]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

{charge => {iterations => 4, strength => -30}, collision => {iterations => 4, radius => 50}, link => {distance => 30}}

------

## Highlight connected components

Make a grid graph with randomly directed edges:

In [61]:
my $g = Graph::Grid.new(10, 20, :!directed);
my $g2 = $g.directed-graph(method => 'random');

Graph(vertexes => 200, edges => 370, directed => True)

Find the connected components in the graph:

In [62]:
my @components = $g2.connected-components.grep(*.elems - 1);
@components».elems;

(69 16 13 6 6 4 4)

Highlight the connected components in a graph:

In [63]:
#% js
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_').reverse».Int });

$g2.edges(:dataset) ==> 
js-d3-graph-plot(
    :%vertex-coordinates,
    highlight => @components,
    directed => $g2.directed,
    :$background, 
    :$title-color, 
    width => 1000, 
    height => 500, 
    vertex-label-color => 'none',
    :$edge-thickness,
    vertex-size => 12, 
    force => {charge => {strength => -200, iterations => 2}, collision => {radius => 30, iterations => 6}, link => {minDistance => 1}}
)

In [64]:
#%html
$g2.vertex-coordinates = %vertex-coordinates;
$g2.dot( 
    highlight => @components,
    :!node-labels,
    node-shape => 'square', 
    node-width => 0.75, 
    edge-thickness => 4, 
    edge-color => 'Gray',
    size => '10,6!',
    engine => 'neato' 
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 11.0.0 (20240428.1522)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 4_13 -->
 
 4_13 
 
 
<!-- 4_14 -->
 
 4_14 
 
 
<!-- 4_13->4_14 -->
 
 4_13->4_14 
 
 
 
<!-- 5_13 -->
 
 5_13 
 
 
<!-- 4_13->5_13 -->
 
 4_13->5_13 
 
 
 
<!-- 4_12 -->
 
 4_12 
 
 
<!-- 4_13->4_12 -->
 
 4_13->4_12 
 
 
 
<!-- 4_8 -->
 
 4_8 
 
 
<!-- 5_8 -->
 
 5_8 
 
 
<!-- 4_8->5_8 -->
 
 4_8->5_8 
 
 
 
<!-- 3_8 -->
 
 3_8 
 
 
<!-- 4_8->3_8 -->
 
 4_8->3_8 
 
 
 
<!-- 4_15 -->
 
 4_15 
 
 
<!-- 4_14->4_15 -->
 
 4_14->4_15 
 
 
 
<!-- 6_19 -->
 
 6_19 
 
 
<!-- 2_3 -->
 
 2_3 
 
 
<!-- 2_4 -->
 
 2_4 
 
 
<!-- 2_3->2_4 -->
 
 2_3->2_4 
 
 
 
<!-- 1_3 -->
 
 1_3 
 
 
<!-- 2_3->1_3 -->
 
 2_3->1_3 
 
 
 
<!-- 1_8 -->
 
 1_8 
 
 
<!-- 0_8 -->
 
 0_8 
 
 
<!-- 1_8->0_8 -->
 
 1_8->0_8 
 
 
 
<!-- 2_8 -->
 
 2_8 
 
 
<!-- 1_8->2_8 -->
 
 1_8->2_8 
 
 
 
<!-- 5_14 -->
 
 5_14 
 
 
<!-- 5_14->4_14 -->
 
 5_14->4_14 
 
 
 
<!-- 6_13 -->
 
 6_13 
 
 
<!-- 6_14 -->
 
 6_14 
 
 
<!-- 6_13->6_14 -->
 
 6_13->6_14 
 
 
 
<!-- 6_12 -->
 
 6_12 
 
 
<!-- 6_13->6_12 -->
 
 6_13->6_12 
 
 
 
<!-- 5_15 -->
 
 5_15 
 
 
<!-- 4_15->5_15 -->
 
 4_15->5_15 
 
 
 
<!-- 3_4 -->
 
 3_4 
 
 
<!-- 2_4->3_4 -->
 
 2_4->3_4 
 
 
 
<!-- 1_4 -->
 
 1_4 
 
 
<!-- 2_4->1_4 -->
 
 2_4->1_4 
 
 
 
<!-- 2_5 -->
 
 2_5 
 
 
<!-- 2_4->2_5 -->
 
 2_4->2_5 
 
 
 
<!-- 0_3 -->
 
 0_3 
 
 
<!-- 0_4 -->
 
 0_4 
 
 
<!-- 0_3->0_4 -->
 
 0_3->0_4 
 
 
 
<!-- 0_2 -->
 
 0_2 
 
 
<!-- 0_3->0_2 -->
 
 0_3->0_2 
 
 
 
<!-- 0_3->1_3 -->
 
 0_3->1_3 
 
 
 
<!-- 7_13 -->
 
 7_13 
 
 
<!-- 7_13->6_13 -->
 
 7_13->6_13 
 
 
 
<!-- 7_12 -->
 
 7_12 
 
 
<!-- 7_13->7_12 -->
 
 7_13->7_12 
 
 
 
<!-- 7_14 -->
 
 7_14 
 
 
<!-- 7_13->7_14 -->
 
 7_13->7_14 
 
 
 
<!-- 8_13 -->
 
 8_13 
 
 
<!-- 7_13->8_13 -->
 
 7_13->8_13 
 
 
 
<!-- 5_16 -->
 
 5_16 
 
 
<!-- 4_16 -->
 
 4_16 
 
 
<!-- 5_16->4_16 -->
 
 5_16->4_16 
 
 
 
<!-- 9_1 -->
 
 9_1 
 
 
<!-- 8_1 -->
 
 8_1 
 
 
<!-- 9_1->8_1 -->
 
 9_1->8_1 
 
 
 
<!-- 2_0 -->
 
 2_0 
 
 
<!-- 1_0 -->
 
 1_0 
 
 
<!-- 2_0->1_0 -->
 
 2_0->1_0 
 
 
 
<!-- 6_6 -->
 
 6_6 
 
 
<!-- 1_17 -->
 
 1_17 
 
 
<!-- 2_17 -->
 
 2_17 
 
 
<!-- 1_17->2_17 -->
 
 1_17->2_17 
 
 
 
<!-- 0_17 -->
 
 0_17 
 
 
<!-- 1_17->0_17 -->
 
 1_17->0_17 
 
 
 
<!-- 8_12 -->
 
 8_12 
 
 
<!-- 8_11 -->
 
 8_11 
 
 
<!-- 8_12->8_11 -->
 
 8_12->8_11 
 
 
 
<!-- 8_12->8_13 -->
 
 8_12->8_13 
 
 
 
<!-- 5_9 -->
 
 5_9 
 
 
<!-- 5_8->5_9 -->
 
 5_8->5_9 
 
 
 
<!-- 5_7 -->
 
 5_7 
 
 
<!-- 5_8->5_7 -->
 
 5_8->5_7 
 
 
 
<!-- 9_3 -->
 
 9_3 
 
 
<!-- 9_4 -->
 
 9_4 
 
 
<!-- 9_3->9_4 -->
 
 9_3->9_4 
 
 
 
<!-- 3_10 -->
 
 3_10 
 
 
<!-- 4_10 -->
 
 4_10 
 
 
<!-- 3_10->4_10 -->
 
 3_10->4_10 
 
 
 
<!-- 3_11 -->
 
 3_11 
 
 
<!-- 3_10->3_11 -->
 
 3_10->3_11 
 
 
 
<!-- 2_10 -->
 
 2_10 
 
 
<!-- 3_10->2_10 -->
 
 3_10->2_10 
 
 
 
<!-- 6_2 -->
 
 6_2 
 
 
<!-- 6_3 -->
 
 6_3 
 
 
<!-- 6_2->6_3 -->
 
 6_2->6_3 
 
 
 
<!-- 7_2 -->
 
 7_2 
 
 
<!-- 6_2->7_2 -->
 
 6_2->7_2 
 
 
 
<!-- 7_15 -->
 
 7_15 
 
 
<!-- 6_15 -->
 
 6_15 
 
 
<!-- 7_15->6_15 -->
 
 7_15->6_15 
 
 
 
<!-- 8_15 -->
 
 8_15 
 
 
<!-- 7_15->8_15 -->
 
 7_15->8_15 
 
 
 
<!-- 2_18 -->
 
 2_18 
 
 
<!-- 2_19 -->
 
 2_19 
 
 
<!-- 2_18->2_19 -->
 
 2_18->2_19 
 
 
 
<!-- 3_18 -->
 
 3_18 
 
 
<!-- 2_18->3_18 -->
 
 2_18->3_18 
 
 
 
<!-- 0_0 -->
 
 0_0 
 
 
<!-- 1_0->0_0 -->
 
 1_0->0_0 
 
 
 
<!-- 1_16 -->
 
 1_16 
 
 
<!-- 1_16->1_17 -->
 
 1_16->1_17 
 
 
 
<!-- 4_5 -->
 
 4_5 
 
 
<!-- 5_5 -->
 
 5_5 
 
 
<!-- 4_5->5_5 -->
 
 4_5->5_5 
 
 
 
<!-- 4_4 -->
 
 4_4 
 
 
<!-- 4_5->4_4 -->
 
 4_5->4_4 
 
 
 
<!-- 4_6 -->
 
 4_6 
 
 
<!-- 4_5->4_6 -->
 
 4_5->4_6 
 
 
 
<!-- 7_4 -->
 
 7_4 
 
 
<!-- 8_4 -->
 
 8_4 
 
 
<!-- 7_4->8_4 -->
 
 7_4->8_4 
 
 
 
<!-- 7_3 -->
 
 7_3 
 
 
<!-- 7_4->7_3 -->
 
 7_4->7_3 
 
 
 
<!-- 8_5 -->
 
 8_5 
 
 
<!-- 8_4->8_5 -->

--------

## Lily pond frog jumping

A frog in a lily pond is able to jump 0.75 meters to get from one of the 25 lily pads to another. Model the frog's jumping network from the lily leaf density and [`SpatialGraphDistribution`](http://reference.wolfram.com/language/ref/SpatialGraphDistribution.html):

In [65]:
# Assumming the pond is 10x10 meters.
my $g = Graph::Random.new(Graph::Distribution::Spatial.new(30, 0.75 / 10))

Graph(vertexes => 30, edges => 5, directed => False)

In [66]:
my @points = [
    |random-variate(BinormalDistribution.new([0,0], [1,1], 0), 10),
    |random-variate(BinormalDistribution.new([-1,4], [1,1], -1/2), 10),
    |random-variate(BinormalDistribution.new([4,4], [1,1], 1/3), 10)
];

@points.elems

30

In [67]:
my $g = Graph::Random.new(Graph::Distribution::Spatial.new(@points.elems, 0));
$g.vertex-coordinates = (@points.kv).Hash;
for ($g.vertex-coordinates.keys X $g.vertex-coordinates.keys) -> $p {
    if $p.head ne $p.tail && euclidean-distance($g.vertex-coordinates{$p.head}, $g.vertex-coordinates{$p.tail}) < 1.5 {
        $g.add-edge($p.head, $p.tail)
    }
}

(Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges => 57, directed => False) Graph(vertexes => 30, edges

In [68]:
#% js
$g.edges(:dataset) ==> 
js-d3-graph-plot(
    highlight => $g.connected-components,
    vertex-coordinates => $g.vertex-coordinates,
    directed => $g.directed,
    background => '#143566', 
    :$title-color, 
    width => 600, 
    height => 600, 
    edge-thickness => 0,
    edge-color => 'Silver',
    vertex-size => 8,
    vertex-color => 'Green',
    vertex-label-color => 'none',
    force => {charge => {strength => -200, iterations => 2}, collision => {radius => 30, iterations => 6}, link => {minDistance => 1}}
)

------

## Topological sorting (via Tarjan's algorithm)

Tarjan's strongly connected components algorithm can be used to find topological sorting. 

Here is a graph from Wikipedia's article ["Topological sorting"](https://en.wikipedia.org/wiki/Topological_sorting):

In [69]:
my @edges = '1_1' => '2_1', '1_2' => '2_1', '2_1' => '3_1', '2_1' => '3_2', '2_1' => '3_3', '1_2' => '2_2', '1_3' => '2_2', '1_3' => '2_2', '2_2' => '3_2';
@edges .= map({ $_.key.Str => $_.value.Str }); 
my $g = Graph.new(@edges, :directed);

my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_').reverse».Int });


{1_1 => (1 1), 1_2 => (2 1), 1_3 => (3 1), 2_1 => (1 2), 2_2 => (2 2), 3_1 => (1 3), 3_2 => (2 3), 3_3 => (3 3)}

In [70]:
#% js
$g.edges(:dataset) ==> 
js-d3-graph-plot(
    #:%vertex-coordinates,
    directed => $g.directed,
    :$background, 
    :$title-color, 
    width => 600, 
    height => 400, 
    :$edge-thickness,
    vertex-size => 12, 
    force => {charge => {strength => -200, iterations => 2}, collision => {radius => 30, iterations => 6}, link => {minDistance => 1}}
)

In [71]:
$g.topological-sort

(1_3 1_2 2_2 1_1 2_1 3_1 3_3 3_2)